In [87]:
import folium, requests
import pandas as pd
import random

df_gist = pd.read_csv('./gist.csv')
app_key = "l7xx19b0512ba5fd4587992aabcae1c10d9b"

# folium map visualize, center: gist
gist_lat = '35.22956444643729'
gist_lon = '126.84774671246136'

m = folium.Map(location=[gist_lat, gist_lon],
                zoom_start=15,
                width='100%',
                height='100%',
                )

folium.Marker([gist_lat, gist_lon],
                popup="gist hub",
                icon=folium.Icon(color='red',icon='star'),
                tooltip="gist hub").add_to(m)

# latitude, logitude of hub and shops
locations = []
locations.append((gist_lat, gist_lon))

# amount of orders for each shops
orders = [random.randint(0, 20) for i in range(1 + len(df_gist))]     # add depot
orders[0] = 0   # 0 for depot

print(orders)

[0, 2, 12, 0, 14, 19, 4, 17, 18, 2, 11, 1, 2, 14, 9, 2, 7, 15, 7, 20, 18, 4, 0, 5, 17, 6, 7, 1, 15, 14, 14, 17, 0, 11, 8, 4, 1, 17, 19, 4, 10, 10]


In [88]:
# tmap geocoding api 호출
url = "https://apis.openapi.sk.com/tmap/geo/fullAddrGeo?version=1&format=json&callback=result"

for index, row in df_gist.iterrows():

    shop_name = row["name"]
    shop_addr = row["address"]

    r = requests.get(url, params={
        "appKey" : app_key,
        "coordType" : "WGS84GEO",
        "fullAddr": shop_addr
    })

    res = r.json().get('coordinateInfo').get('coordinate')[0]
    
    latitude = ''
    longitude = ''
    
    if (res.get('latEntr') != ''):              # 구주소 입구점 위*경도
        latitude = res.get('latEntr')
        longitude = res.get('lonEntr')
    elif (res.get('newLatEntr') != ''):         # 새주소 입구점 위*경도
        latitude = res.get('newLatEntr')
        longitude = res.get('newLonEntr')
    elif (res.get('lat') != ''):                # 구주소 위*경도
        latitude = res.get('lat')
        longitude = res.get('lon')
    elif (res.get('newLat') != ''):             # 새주소 위*경도
        latitude = res.get('newLat')
        longitude = res.get('newLon')
    else:
        continue

    locations.append((latitude, longitude))
    print(shop_name, shop_addr, latitude, longitude)

    # todo. 동일한 좌표의 marker가 한개만 찍히는 문제 해결하기
    folium.Marker([latitude, longitude],
                popup=shop_name,
                tooltip=shop_name).add_to(m)

print(len(locations))
m


한솥도시락 광주광역시 광산구 임방울대로 727-6 35.218452 126.837765
맘스터치 광주광역시 광산구 첨단중앙로 96 35.213924 126.843209
킹스샌드&샐러드 광주광역시 광산구 첨단내촌로57번길 6 35.211897 126.836793
달콤 광주광역시 광산구 첨단내촌로57번길 6 35.211897 126.836793
마마죽&스프 광주광역시 광산구 첨단내촌로57번길 6 35.211897 126.836793
김밥랜드 광주광역시 광산구 월계동 758-4 35.221201 126.840571
무양식탁 광주광역시 광산구 첨단내촌로36번길, 28 35.210008 126.838904
2학식당 광주광역시 북구 첨단과기로 123 35.225034 126.840154
또아식빵 광주광역시 광산구 첨단중앙로 106번길 23 35.214924 126.844237
기가찬국밥 광주광역시 광산구 첨단중앙로 106번길 30 35.21473 126.844765
원할머니보쌈 광주광역시 광산구 첨단중앙로182번길 24 35.221757 126.84407
유생촌 광주광역시 광산구 월계동 881-1 35.216063 126.848265
이마트24 광주광역시 광산구 첨단중앙로 96 35.213924 126.843209
통큰고기 통큰비빔밥 광주광역시 광산구 월계동826-3 35.212924 126.841654
마초떡볶이 광주광역시 광산구 월계동 851-16 35.20898 126.841237
일계양득 광주광역시 광산구 월계동 851-16 35.20898 126.841237
국이랑 찌개랑 광주광역시 광산구 월계동831-6번지 1층 35.211563 126.839821
도시락데이 광주광역시 광산구 월계동831-6번지 1층 35.211563 126.839821
김밥나라 광주광역시 광산구 임방울대로 826번길 35.215146 126.848515
설빙 광주광역시 광산구 첨단중앙로 104 2층 35.214758 126.843571
김정호 깐풍기 광주광역시 광산구 산월로 33 35.20

In [ ]:
import json

# Tmap 타임머신 자동차 길 안내 api
# 패턴 교통정보를 이용하여 미래의 시간에 차량의 경로를 탐색

url = "https://apis.openapi.sk.com/tmap/routes/prediction?version=1&format=json" 	# qeury &totalValue=2 추가하면 기본 정보만
prediction_time = "2022-06-30T11:30:00+0900"
departure_addr = ["35.218452", "126.837765"]
dest_addr = [gist_lat, gist_lon]

headers = {
	"appKey" : app_key,
	"Content-Type" : "application/json"
}

datas = {
	"routesInfo" : {
		"departure" : {
			"name" : "test1",
			"lat" : departure_addr[0],
			"lon" : departure_addr[1]
		},
		"destination" : {
			"name" : "test2",
			"lat" : dest_addr[0],
			"lon" : dest_addr[1]
		},
		"predictionType" : "arrival", # todo. 도착시간 예측 길 안내 -> 출발로 변경?
		"predictionTime" : prediction_time,
		# "wayPoints" : {
		# 	"wayPoint" : [ {
		# 		"lat" : departure_addr[0]
		# 		"lon" : departure_addr[1],
		# 	}, {
		# 		"lat" : dest_addr[0]
		# 		"lon" : dest_addr[1],
		# 	} ]
		# },
		"searchOption" : "00"   # todo. 우선순위: 교통최적+추천 -> 02.교통최적+최소시간?
	}
}

r = requests.post(url, headers=headers, data=json.dumps(datas))

res = r.json().get('features')[0].get('properties')
pred_distance = res.get('totalDistance')	# meter
pred_time = res.get('totalTime')			# sec

In [90]:
## test : folium poly line 그리기

response = r.json().get('features')

# print(response)
points = []

for i in range (len(response)):
    ptype = response[i].get('geometry').get('type')
    path = response[i].get('geometry').get('coordinates')
    # print(path)
    if (ptype == "Point"):
        points.append([path[1], path[0]])       # response = [lon, lat]로 들어옴
    elif (ptype == "LineString"):
        for j in range (len(path)):
            line = path[j]
            points.append([path[j][1], path[j][0]])

folium.PolyLine(points, color='red').add_to(m)

m

In [104]:
import time

# delete node with zero order

temp_locations = []
temp_orders = []
for i in range (len(orders)):
    if (i!=0 and orders[i] == 0):
        continue
    else:
        temp_locations.append(locations[i])
        temp_orders.append(orders[i])

print(len(locations))
locations = temp_locations
orders = temp_orders

# 일단 단순히 직선거리만 고려!

def create_data_model():
    data = {}
    
    # Tmap 타임머신 자동차 길 안내 api
    # 패턴 교통정보를 이용하여 미래의 시간에 차량의 경로를 탐색

    url = "https://apis.openapi.sk.com/tmap/routes/prediction?version=1&format=json&totalValue=2" 	# 기본 정보만 요청
    prediction_time = "2022-06-30T11:30:00+0900"	# todo. 출발 시각 바꾸기 (점/저)

    headers = {
        "appKey" : app_key,
        "Content-Type" : "application/json"
    }

    dist_matrix = []
    time_matrix = []
    # todo. ortools distance_matrix가 asymmetric해도 되나?
    for i in range (len(locations)):        # 출발
        temp_dist_list = []     # todo. dist 계산 방향성? 출->도?
        temp_time_list = []

        # todo. tmap api 요청 최대 건수 초과 -> symmetric한 matrix로 만들기!
        
        for j in range (len(locations)):    # 도착    
            if (i!=j):
                datas = {
                    "routesInfo" : {
                        "departure" : {
                            "name" : "test1",
                            "lat" : locations[i][0],
                            "lon" : locations[i][1]
                        },
                        "destination" : {
                            "name" : "test2",
                            "lat" : locations[j][0],
                            "lon" : locations[j][1]
                        },
                        "predictionType" : "arrival", # todo. 도착시간 예측 길 안내 -> 출발로 변경?
                        "predictionTime" : prediction_time,
                        "searchOption" : "00"   # todo. 우선순위: 교통최적+추천 -> 02.교통최적+최소시간?
                    }
                }

                r = requests.post(url, headers=headers, data=json.dumps(datas))
                print("i: ", i, "j: ", j, r.json())
                res = r.json().get('features')[0].get('properties')
                pred_distance = res.get('totalDistance')	# meter
                pred_time = res.get('totalTime')			# sec

                time.sleep(0.5)       # todo. tmap 초당 처리 건수 초과 방지
            else:
                pred_distance = 0
                pred_time = 0

            temp_dist_list.append(pred_distance)
            temp_time_list.append(pred_time)

        dist_matrix.append(temp_dist_list)
        time_matrix.append(temp_time_list)
    
    print("dist_matrix", dist_matrix)
    print("time_matrix", time_matrix)

    data['distance_matrix'] = [[abs(float(locations[i][0])-float(locations[j][0]))+abs(float(locations[i][1])-float(locations[j][1])) for j in range(len(locations))] for i in range(len(locations))]
    
    # 임의의 위치에서 시작하게 하려면 depot->node로의 거리를 0으로 두면 된다.
    data['distance_matrix'][0] = [0 for i in range(len(locations))]
    data['orders'] = orders
    
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

create_data_model()

39
i:  0 j:  1 {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'totalDistance': 2125, 'totalTime': 414, 'totalFare': 0, 'taxiFare': 3700, 'departureTime': '2022-06-30T11:30:00+0900', 'arrivalTime': '2022-06-30T11:36:54+0900'}}]}
i:  0 j:  2 {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'totalDistance': 2358, 'totalTime': 431, 'totalFare': 0, 'taxiFare': 3700, 'departureTime': '2022-06-30T11:30:00+0900', 'arrivalTime': '2022-06-30T11:37:11+0900'}}]}
i:  0 j:  3 {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'totalDistance': 3126, 'totalTime': 649, 'totalFare': 0, 'taxiFare': 4800, 'departureTime': '2022-06-30T11:30:00+0900', 'arrivalTime': '2022-06-30T11:40:49+0900'}}]}
i:  0 j:  4 {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'totalDistance': 3126, 'totalTime': 649, 'totalFare': 0, 'taxiFare': 4800, 'departureTime': '2022-06-30T11:30:00+0900', 'arrivalTime': '202

TypeError: 'NoneType' object is not subscriptable

In [92]:
def print_solution(data, manager, routing, solution):
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_order = 0
        index = solution.Value(routing.NextVar(index))

        root_arr_x = []
        root_arr_y = []

        while not routing.IsEnd(index):
            root_arr_x.append(locations[manager.IndexToNode(index)][0])
            root_arr_y.append(locations[manager.IndexToNode(index)][1])
            # plt.text(locations[manager.IndexToNode(index)][0], locations[manager.IndexToNode(index)][1], index)

            node_index = manager.IndexToNode(index)
            route_order += data['orders'][node_index]
            # plt.text(locations[manager.IndexToNode(index)][0]+30, locations[manager.IndexToNode(index)][1]+30, data['orders'][node_index], c="dodgerblue")

            plan_output += ' {0} load ({1}) -> '.format(node_index, route_order)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            # print(route_distance)

        root_arr_x.append(locations[manager.IndexToNode(index)][0])
        root_arr_y.append(locations[manager.IndexToNode(index)][1])
        # plt.text(locations[manager.IndexToNode(index)][0], locations[manager.IndexToNode(index)][1], index)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        # plt.plot(root_arr_x, root_arr_y)
        
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    
    # plt.gca().invert_yaxis()
    # plt.show()
    print('Maximum of the route distances: {}m'.format(max_route_distance))

In [93]:
from ortools.constraint_solver import routing_enums_pb2, pywrapcp

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    # argument: (num of total locations including depot(출발지, 종점), num of vehicles, node of depot)
    # geek -> 상점 수, 드라이버 수, hub의 node
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    # geek -> 실제 상점 간 거리 (교통 상황 반영되어야 함.)
    # geek -> 각 node에서 적재하는 데 걸리는 시간 보정 필요
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
        #  + data['orders'][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance -> geek: max time per vehicle로 변경
        True,  # start cumul to zero -> geek: Don't force start cumul to zero.??
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # routing.AddVariableMaximizedByFinalizer(
    #     distance_dimension.CumulVar(routing.Start(i))
    # )

    # routing.AddWeightedVariableMinimizedByFinalizer(
    #     routing.NextVar(10)
    #     , 5000
    # )
    
    # routing.AddWeightedVariableTargetToFinalizer(IntVar* var, int64_t cost)
    # routing.AddWeightedVariableMaximizedByFinalizer()
    # routing.AddWeightedVariableMinimizedByFinalizer()

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')

main()

Objective: 0
Route for vehicle 0:
0
Distance of the route: 0m

Route for vehicle 1:
0
Distance of the route: 0m

Route for vehicle 2:
0
Distance of the route: 0m

Route for vehicle 3:
 38 load (10) ->  37 load (20) ->  36 load (24) ->  35 load (43) ->  34 load (60) ->  33 load (61) ->  32 load (65) ->  31 load (73) ->  30 load (84) ->  29 load (101) ->  28 load (115) ->  27 load (129) ->  26 load (144) ->  25 load (145) ->  24 load (152) ->  23 load (158) ->  22 load (175) ->  21 load (180) ->  20 load (184) ->  19 load (202) ->  18 load (222) ->  17 load (229) ->  16 load (244) ->  15 load (251) ->  14 load (253) ->  13 load (262) ->  12 load (276) ->  11 load (278) ->  10 load (279) ->  9 load (290) ->  8 load (292) ->  7 load (310) ->  6 load (327) ->  5 load (331) ->  4 load (350) ->  3 load (364) ->  2 load (376) ->  1 load (378) -> 0
Distance of the route: 0m

Maximum of the route distances: 0m


In [94]:
print(locations)
print(len(locations))

print(len(orders))

print(len(df_gist))

[('35.22956444643729', '126.84774671246136'), ('35.218452', '126.837765'), ('35.213924', '126.843209'), ('35.211897', '126.836793'), ('35.211897', '126.836793'), ('35.221201', '126.840571'), ('35.210008', '126.838904'), ('35.225034', '126.840154'), ('35.214924', '126.844237'), ('35.21473', '126.844765'), ('35.221757', '126.84407'), ('35.216063', '126.848265'), ('35.213924', '126.843209'), ('35.212924', '126.841654'), ('35.20898', '126.841237'), ('35.20898', '126.841237'), ('35.211563', '126.839821'), ('35.211563', '126.839821'), ('35.215146', '126.848515'), ('35.214758', '126.843571'), ('35.208953', '126.840821'), ('35.209036', '126.835877'), ('35.209758', '126.839488'), ('35.214646', '126.84882'), ('35.209036', '126.840015'), ('35.215146', '126.845098'), ('35.214646', '126.84882'), ('35.214646', '126.84907'), ('35.214646', '126.84882'), ('35.214646', '126.84882'), ('35.220035', '126.842487'), ('35.213924', '126.845793'), ('35.214119', '126.84607'), ('35.219285', '126.842654'), ('35.21